In [ ]:
# This produces the dataframe for WR

In [ ]:
## Notes on the NFL Library ##
# the NFL python library seem to not work on Tuesday probably due to updates
# best to run this on a day other than Tues.

In [ ]:
## REQUIRED ACTIONS - Include in a README doc ## 
# modify the season start date in the 'get_current_week' function
#

In [55]:
# import the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import glob
from IPython.display import display, HTML
from datetime import datetime
import nfl_data_py as nfl
import os
import re

In [3]:
# Set Pandas options to display all columns in a single row without wrapping
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [67]:
# Function to calculate the current week of the NFL season
def get_current_week():
    current_date = datetime.now()
    season_start_date = datetime(2024, 9, 4)  # Update for the season start
    current_week = ((current_date - season_start_date).days // 7) + 1
    return current_week

# Define the current NFL year, week, and season type
current_year = datetime.now().year
current_week = get_current_week()
seasontype = 2 if current_week <= 18 else 3  # Regular season or playoffs

In [70]:
# define the years to pull
# nfl.import_weekly_data(years, columns, downcast)
def get_year_range(current_year, current_week, start_year=2017):
    if current_week <= 18:  # Regular season
        return list(range(start_year, current_year + 1))
    else:  # Playoffs
        return list(range(start_year, current_year))

# Use the function
years = get_year_range(current_year, current_week)

In [95]:
# define the base columns. 
base_columns = [
    'season', 'season_type', 'week', 'player_id', 'player_name', 
    'position', 'position_group', 'recent_team'
]

In [93]:
# import the player ids from nfl.import_ids() - without parameters
ids_data = nfl.import_ids()
# ids_data

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,16579,49d4b627-3f19-49a8-ae33-f539e0fb1f88,23084.0,00-0039918,NaN,11560.0,NaN,4431611.0,40900.0,NaN,26695775.0,WillCa03,caleb-williams-3,17695.0,NaN,1550.0,40900.0,0.0,NaN,1286445.0,Caleb Williams,caleb williams,QB,CHI,2001-11-18,23.1,2024.0,1.0,1.0,1.0,NaN,73.0,215.0,USC,2024
1,16580,d0c0630e-8925-4b81-a32b-e4146d95f01f,23046.0,00-0039851,NaN,11564.0,NaN,4431452.0,40881.0,NaN,26695757.0,MayeDr00,drake-maye-1,17673.0,NaN,1551.0,40881.0,0.0,NaN,1286803.0,Drake Maye,drake maye,QB,NEP,2002-08-30,22.3,2024.0,1.0,3.0,3.0,NaN,76.0,225.0,North Carolina,2024
2,16581,bab8c65a-9fff-410d-8e34-dc2f636d890e,22902.0,00-0039910,NaN,11566.0,NaN,4426348.0,40896.0,NaN,3123337.0,DaniJa02,jayden-daniels-1,17692.0,NaN,1552.0,40896.0,0.0,NaN,1161228.0,Jayden Daniels,jayden daniels,QB,WAS,2000-12-18,24.0,2024.0,1.0,2.0,2.0,NaN,76.0,210.0,LSU,2024
3,16582,f056c8ed-6ee5-492c-af11-3f7e3d97195c,22910.0,00-0039732,NaN,11563.0,NaN,4426338.0,40875.0,NaN,3121193.0,NixxBo00,bo-nix-1,16995.0,NaN,1553.0,40875.0,0.0,NaN,1161899.0,Bo Nix,bo nix,QB,DEN,2000-02-25,24.9,2024.0,1.0,12.0,12.0,NaN,74.0,217.0,Oregon,2024
4,16583,6499a161-b221-4591-b122-d99ece1be64b,22973.0,00-0039917,NaN,11559.0,NaN,4360423.0,40889.0,NaN,2963496.0,PeniMi00,michael-penix-jr-1,17700.0,NaN,1554.0,40889.0,0.0,NaN,1107478.0,Michael Penix Jr.,michael penix,QB,ATL,2000-05-08,24.7,2024.0,1.0,8.0,8.0,NaN,74.0,220.0,Washington,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11772,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11773,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11774,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
11775,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bubby Brister,bubby brister,QB,KCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024


In [102]:
# import the weekly data from nfl.import_weekly_data(years, columns, downcast)
weekly_data = nfl.import_weekly_data(
    years=years,
    columns=base_columns
)

Downcasting floats.


In [105]:
## Output: a dataframe of all nfl athletes and their associated ids for each week of each season since 2017

# Merge the two dataframes on 'player_id' and 'gsis_id'
# Align column names for merging
ids_data = ids_data.rename(columns={'gsis_id': 'player_id'})  
id_dataframe = pd.merge(weekly_data, ids_data, on='player_id', how='inner')

# Assign the resulting dataframe to a variable
all_players_id_data = id_dataframe

# Display the resulting ID dataframe
display(all_players_id_data)

,season,season_type,week,player_id,player_name,position_x,position_group,recent_team,mfl_id,sportradar_id,fantasypros_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position_y,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,2017,REG,1,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,QB,FA,1977-08-03,47.4,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,2024
1,2017,REG,2,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,QB,FA,1977-08-03,47.4,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,2024
2,2017,REG,3,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,QB,FA,1977-08-03,47.4,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,2024
3,2017,REG,4,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,QB,FA,1977-08-03,47.4,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,2024
4,2017,REG,5,00-0019596,T.Brady,QB,QB,NE,5848,41c44740-d0f6-44ab-8347-3b5d515e5ecf,9603.0,698.0,167.0,tombrady/2504211,2330.0,5228.0,309.0,187741.0,BradTo00,tom-brady-1,1350.0,1163.0,NaN,5228.0,25347.0,4314.0,25347.0,Tom Brady,tom brady,QB,FA,1977-08-03,47.4,2000.0,6.0,33.0,199.0,NaN,76.0,225.0,Michigan,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43421,2024,REG,10,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,RB,ARI,2002-07-23,22.5,2024.0,3.0,2.0,66.0,NaN,73.0,223.0,Florida State,2024
43422,2024,REG,12,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,RB,ARI,2002-07-23,22.5,2024.0,3.0,2.0,66.0,NaN,73.0,223.0,Florida State,2024
43423,2024,REG,13,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,RB,ARI,2002-07-23,22.5,2024.0,3.0,2.0,66.0,NaN,73.0,223.0,Florida State,2024
43424,2024,REG,14,00-0039921,T.Benson,RB,RB,ARI,16594,0c1253f2-1822-445d-8590-fea6fd32703f,25388.0,NaN,11589.0,NaN,4429275.0,40958.0,NaN,3176915.0,BensTr01,trey-benson-2,17767.0,NaN,1565.0,40958.0,0.0,NaN,1248031.0,Trey Benson,trey benson,RB,ARI,2002-07-23,22.5,2024.0,3.0,2.0,66.0,NaN,73.0,223.0,Florida State,2024
